In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('mel_features_dataset.csv')

In [3]:
X = df['Mel_Features'].values
y = df['Ragas'].values

In [4]:
X = np.array(df['Mel_Features'].tolist())
y = np.array(df['Ragas'].tolist())

In [5]:
print("Shape of X before reshape:", X.shape)
print("Shape of y before encode:", y.shape)

Shape of X before reshape: (92,)
Shape of y before encode: (92,)


In [6]:
le=LabelEncoder()
ti=le.fit_transform(y)
y_mel=to_categorical(ti)

In [7]:
y_mel

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1

In [8]:
X_train,X_test,y_train,y_test=train_test_split(X,y_mel,test_size=0.2,random_state=127)

In [9]:
num_classes = len(y_mel[0])
print(num_classes)

3


In [10]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
X_train_flat = np.array([x.flatten() for x in X_train])

In [11]:
print("Shape of X_train:", X_train_flat.shape)

Shape of X_train: (73, 1)


In [12]:
y_train_numerical = np.argmax(y_train, axis=1)
y_test_numerical = np.argmax(y_test, axis=1)
y_train_encoded = to_categorical(y_train_numerical, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_numerical, num_classes=num_classes)

In [13]:
print(X_train[0])

['[[-53.29912567 -37.78041077 -27.55327797 ... -80.         -80.\n  -80.        ]\n [-54.55303574 -36.44094086 -27.5207653  ... -80.         -80.\n  -80.        ]\n [-55.10224152 -36.41986847 -29.47444153 ... -80.         -80.\n  -80.        ]\n ...\n [-53.42518234 -37.04436111 -24.7610302  ... -80.         -80.\n  -80.        ]\n [-57.92776489 -36.66670227 -24.26868057 ... -80.         -80.\n  -80.        ]\n [-33.79752731 -30.21833801 -21.74354172 ... -80.         -80.\n  -80.        ]]']


In [14]:
print(X[0])

[[-61.36026764 -56.3211441  -39.66576004 ... -80.         -80.
  -80.        ]
 [-62.71931458 -53.63751221 -39.98358154 ... -80.         -80.
  -80.        ]
 [-60.93008041 -51.95860291 -40.66412354 ... -80.         -80.
  -80.        ]
 ...
 [-62.52468491 -56.15249634 -40.87715149 ... -77.84117126 -80.
  -80.        ]
 [-47.95288467 -45.6871109  -39.36861038 ... -79.81661224 -80.
  -80.        ]
 [-37.45179749 -36.0095253  -33.8965683  ... -77.08987427 -78.45914459
  -80.        ]]


In [20]:
list_of_arrays = []
for item in X_train:
    cleaned_string = str(item).strip('[]').replace('\n', '').strip()
    if cleaned_string:
        try:
            array_elements = [float(x) for x in cleaned_string.split(' ') if x]
            list_of_arrays.append(np.array(array_elements))
        except ValueError as e:
            print(f"Could not convert string to float for item: '{item}' - Error: {e}")
            list_of_arrays.append(np.array([]))
    else:
        list_of_arrays.append(np.array([]))

Could not convert string to float for item: '['[[-53.29912567 -37.78041077 -27.55327797 ... -80.         -80.\n  -80.        ]\n [-54.55303574 -36.44094086 -27.5207653  ... -80.         -80.\n  -80.        ]\n [-55.10224152 -36.41986847 -29.47444153 ... -80.         -80.\n  -80.        ]\n ...\n [-53.42518234 -37.04436111 -24.7610302  ... -80.         -80.\n  -80.        ]\n [-57.92776489 -36.66670227 -24.26868057 ... -80.         -80.\n  -80.        ]\n [-33.79752731 -30.21833801 -21.74354172 ... -80.         -80.\n  -80.        ]]']' - Error: could not convert string to float: "'[[-53.29912567"
Could not convert string to float for item: '['[[-34.73913574 -28.62771225 -12.71242619 ... -80.         -80.\n  -80.        ]\n [-35.1524353  -27.95057297 -11.9528656  ... -80.         -80.\n  -80.        ]\n [-29.08322525 -25.07164955 -11.18974876 ... -80.         -80.\n  -80.        ]\n ...\n [-55.69836044 -55.04442596 -50.96155548 ... -80.         -80.\n  -80.        ]\n [-58.9162941  -53.

In [22]:
import ast
import re

list_of_arrays = []
for item in X_train:
    try:
        evaluated_item_str = ast.literal_eval(str(item))
        cleaned_string = str(evaluated_item_str).strip('[]').replace('\n', ' ')
        cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()

        if cleaned_string:
            temp_array = np.fromstring(cleaned_string, sep=' ')
            list_of_arrays.append(temp_array)
        else:
            list_of_arrays.append(np.array([]))

    except (ValueError, SyntaxError) as e:
        print(f"Error processing item: '{item}' - Error: {e}")
        list_of_arrays.append(np.array([]))

<ipython-input-22-f76abdecb0e2>:12: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  temp_array = np.fromstring(cleaned_string, sep=' ')


In [19]:
X_train_float = np.array([np.fromstring(str(element).strip('[]').replace('\n', '').replace(',', ''), sep=' ') for element in X_train])
X_test_float = np.array([np.fromstring(str(element).strip('[]').replace('\n', '').replace(',', ''), sep=' ') for element in X_test])

<ipython-input-19-c2bc923792b7>:1: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  X_train_float = np.array([np.fromstring(str(element).strip('[]').replace('\n', '').replace(',', ''), sep=' ') for element in X_train])
<ipython-input-19-c2bc923792b7>:2: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  X_test_float = np.array([np.fromstring(str(element).strip('[]').replace('\n', '').replace(',', ''), sep=' ') for element in X_test])


In [25]:
print(np.array(list_of_arrays).shape)

(73, 0)


In [ ]:
model_mel = Sequential()
model_mel.add(Conv1D(32, 3, activation='relu', input_shape=(X_train_flat.shape[0], 1)))
model_mel.add(MaxPooling1D(2))
model_mel.add(Conv1D(64, 3, activation='relu'))
model_mel.add(MaxPooling1D(2))
model_mel.add(Flatten())
model_mel.add(Dense(128, activation='relu'))
model_mel.add(Dense(64, activation='relu'))
model_mel.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model_mel = Sequential()
model_mel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X_train_float.shape[1], X_train_float.shape[2], 1)))
model_mel.add(MaxPooling2D((2, 2)))
model_mel.add(Conv2D(64, (3, 3), activation='relu'))
model_mel.add(MaxPooling2D((2, 2)))
model_mel.add(Flatten())
model_mel.add(Dense(128, activation='relu'))
model_mel.add(Dense(64, activation='relu'))
model_mel.add(Dense(num_classes, activation='softmax'))

IndexError: tuple index out of range

In [ ]:
model_mel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_mel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 71, 32)         │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 35, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 33, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 16, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 145,987 (570.26 KB)

 Trainable params: 145,987 (570.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#print("X_train dtype:", X_train_float.dtype)
#print("y_train_encoded dtype:", y_train_encoded.dtype)
#print("X_test dtype:", X_test_float.dtype)
#print("y_test_encoded dtype:", y_test_encoded.dtype)

In [ ]:
history_mel = model_mel.fit(X_train_flat, y_train_encoded, epochs=30, batch_size=32, validation_data=(X_test, y_test_encoded))

ValueError: Invalid dtype: object